# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.common import ML_api
influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


# 1. set param from Front End

In [2]:
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        #step_split일 경우만 past_step과 future_step이 존재
        'data_clean_option' : "false"
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'data_clean_option' : "true",
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    },
}

param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "true",
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        'past_step':4, #step_split일 경우만 past_step과 future_step이 존재
        'future_step':2
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True","num_classes":6}
    },
}

# 2. Backend

In [3]:
# 1. Backend Prameter Setting
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.common import ML_pipeline, tool

# parameter tunning
param = param1

# 관련한 아래 코드 모든 스트링 중 True, None, False 등의 문자열이 있을경우 대소문자 상관 없이 Boolean으로 바르게 변환하도록 수정
# 아래처럼 짤 필요가 없음
# if 'bidirectional' in list(param['model_info']['model_parameter'].keys()):
#     param['model_info']['model_parameter']["bidirectional"] = True if param['model_info']['model_parameter']["bidirectional"] == "true" else False 

# param['data_y_flag'] = True if param['data_y_flag'] == 'true' else False
# param['transform_param']['data_clean_option'] = True if param['transform_param']['data_clean_option'] == "true" else False
#################################

param = ML_api.chagne_type_str_to_bool(param)

param['model_info']['train_parameter']['device'] = device                                                                              
                                                                                         
if param['transform_param']['data_clean_option']:
    param['transform_param']['nan_process_info']= {'type':'num', 'ConsecutiveNanLimit':10, 'totalNaNLimit':100}
    param['transform_param']['max_nan_limit_ratio'] = 0.9
else:
    param['transform_param']['nan_process_info'] = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}
    param['transform_param']['max_nan_limit_ratio'] = 0.5
    
# model name check  
param['model_info']['model_name'] = ML_api.check_model_name(param['model_info']['model_name'],
                                     [param['ingestion_param_X']['ms_name'], param['model_info']['model_purpose'], param['model_info']['model_method']])

In [4]:
# 2. train data meta check
train_data_info = ML_api.get_train_data_meta (mongodb_client, param['ingestion_param_X'])
param['train_data_info'] = train_data_info

In [5]:
# 3. Data Preparation
train_X_array, train_y_array, val_X_array, val_y_array = ML_api.ML_data_preparation(param, influxdb_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_trainX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}
['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
./scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl
['value']
Make New scaler File
window_size: 144 nan_limit_num: 72
(10944, 24) (72, 144, 24)
(76, 1) (72, 1)
window_size: 144 nan_limit_num: 72
(2736, 24) (19, 144, 24)
(19, 1) (19, 1)


In [6]:
# 4.Training
param = ML_api.ML_training(train_X_array,  train_y_array, val_X_array, val_y_array, param)

{'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}
./Models/LSTM_rg/regression_energy_cleanLevel4_trainX_regression_LSTM_rg_/regression_energy_cleanLevel4_trainX/model.pkl
Start training model
[1/5] Training loss: 0.2152	 Validation loss: 0.1540
[2/5] Training loss: 0.1981	 Validation loss: 0.1418
[3/5] Training loss: 0.1745	 Validation loss: 0.1300
[4/5] Training loss: 0.1777	 Validation loss: 0.1185
[5/5] Training loss: 0.1632	 Validation loss: 0.1072

Training complete in 0m 1s


In [7]:
param

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/'},
 'transform_param': {'split_mode': 'window_split',
  'data_clean_option': False,
  'nan_process_info': {'type': 'num',
   'ConsecutiveNanLimit': 10000,
   'totalNaNLimit': 100000},
  'max_nan_limit_ratio': 0.5,
  'future_step': None,
  'past_step': 144},
 'model_info': {'model_purpose': 'regression',
  'data_clean_option': True,
  'model_metho

In [8]:
mongodb_client

In [9]:
# save meta
ml_meta.save_model_meta_data(mongodb_client, param)

========== Data Save Success ==========
======== OK ========


200

In [10]:
param

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/'},
 'transform_param': {'split_mode': 'window_split',
  'data_clean_option': False,
  'nan_process_info': {'type': 'num',
   'ConsecutiveNanLimit': 10000,
   'totalNaNLimit': 100000},
  'max_nan_limit_ratio': 0.5,
  'future_step': None,
  'past_step': 144},
 'model_info': {'model_purpose': 'regression',
  'data_clean_option': True,
  'model_metho